<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week06/pyspark/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

--2022-10-21 06:13:15--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12002039 (11M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar.6’

spark-streaming-kaf 100%[===================>]  11.45M  --.-KB/s    in 0.04s   

2022-10-21 06:13:15 (258 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar.6’ saved [12002039/12002039]



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [ ]:
kafka_topic_name = "my-first-topic"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [ ]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,5) # Show results every 5 seconds

In [ ]:
kvs = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'smallest'
                        })
wc = kvs.map(lambda x: x[1]).flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
wc.pprint()

'''step0 = kvs.map(lambda x: x[1])
#print("step0") # print once
step0.pprint()
step1 = step0.flatMap(lambda line: line.split(' '))
step1.pprint()
step2 = step1.map(lambda word: (word, 1))
step2.pprint()
step3 = step2.reduceByKey(lambda a, b: a+b) #.filter(lambda y: y[1]>5)
step3.pprint()'''

ssc.start()
ssc.awaitTerminationOrTimeout(100)

In [ ]:
ssc.stop()
sc.stop()